Setup

In [1]:
import os
import pandas as pd
from rich.console import Console
from rich.logging import RichHandler
from rich import pretty
import logging
import warnings
import subprocess
import hvac
from cryptography.fernet import Fernet
import base64

DATAFILE="/usr/src/working/data/Questions.xlsx"
ENCRYPTEDFILE="/usr/src/working/data/Questions_encrypted.xlsx"
KEYFILE="/usr/src/working/data/Questions.key"

# Configure logging
logging.disable(logging.WARNING) # disable warnings in the logger
warnings.filterwarnings("ignore") # disable warnings in warnings

logging.basicConfig(level=logging.INFO, format="%(message)s",handlers=[RichHandler()])
console = Console()

Load the decrypted datafile

In [2]:
console.print(f"Loading data from [white]{DATAFILE}[/white]", style="bold magenta")
pd.read_excel(DATAFILE)

Loading data from /usr/src/working/data/Questions.xlsx

,Question,Answer
0,Who is Nic Jackson,Nic Jackson is a thourouly nice chap who works...
1,What is bear blood,Bear blood is a delicous beverage that Ivan pr...


Fetch the encryption key from Vault

In [13]:
client = hvac.Client(
    url=os.environ["VAULT_ADDR"]
)

client.auth.userpass.login(
    username="ops",
    password="password",
)

key = Fernet.generate_key()

fernet = Fernet(key)

with open(DATAFILE,'rb') as file:
    original = file.read()

encrypted = fernet.encrypt(original)

with open(ENCRYPTEDFILE,'wb') as encrypted_file:
    encrypted_file.write(encrypted)


base_key = base64.b64encode(key).decode("ascii")

ciphertext = client.secrets.transit.encrypt_data(
    name="data",
    plaintext=base_key)

with open(KEYFILE, 'wb') as key_file:
    key_file.write(ciphertext['data']['ciphertext'].encode('ascii'))

